In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv
/kaggle/input/my-multi-class-prediction-of-obesity-risk/data.csv


# **肥満リスクの多クラス予測**
(Multi-Class Prediction of Obesity Risk)

# 参考資料
**書籍**


**Kaggle**
- [Obesity! | EDA | XGBoost | Py](https://www.kaggle.com/code/khsamaha/obesity-eda-xgboost-py)

**自分で作成したファイル**
- 銀行解約データセットを使用した二項分類
- 肝硬変の転帰の多クラス予測

**その他**


In [2]:
import numpy as np
import pandas as pd

FILE_PATH = '/kaggle/input/playground-series-s4e2/'
CREATE_DATA = '/kaggle/input/my-multi-class-prediction-of-obesity-risk'


# 確認 (data_import.py)
def file_to_xy(filename):
    data = pd.read_csv(filename, index_col=0)
    print(f'読み込み完了 {filename}')
    
    target_col = 'NObeyesdad'
    
    train = data[:20758].reset_index(drop=True)
    test = data[20758:].reset_index(drop=True).drop(target_col, axis=1) # 目的変数を指定する
    # 目的変数と説明変数に分割
    X = train.drop([target_col], axis=1) # 目的変数を指定する
    y = train[target_col].values # 目的変数を指定する

    return data,test,train,X,y

filename = 'data.csv'
data,test,train,X,y = file_to_xy(CREATE_DATA + '/' + filename)

読み込み完了 /kaggle/input/my-multi-class-prediction-of-obesity-risk/data.csv


/tmp/ipykernel_34/358891333.py:10: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename, index_col=0)


In [17]:
condlist = [train["NObeyesdad"] == "Insufficient_Weight",
            train["NObeyesdad"] == "Normal_Weight",
            train["NObeyesdad"] == "Obesity_Type_I",
            train["NObeyesdad"] == "Obesity_Type_II",
            train["NObeyesdad"] == "Obesity_Type_III",
            train["NObeyesdad"] == "Overweight_Level_I",
            train["NObeyesdad"] == "Overweight_Level_II",
           ]
choicelist = [0,1,2,3,4,5,6]

train["NObeyesdad"] = np.select(condlist, choicelist)

In [19]:
train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,1,1095,933,1839,1,1,265,589,2,0,1491,0,0,1053,2,3,6
1,0,108,128,1376,1,1,265,624,1,0,756,0,751,1068,3,0,1
2,0,108,1040,1253,1,1,206,127,2,0,659,0,587,1322,3,3,0
3,0,522,1035,936,1,1,1053,624,2,0,464,0,1040,806,2,3,4
4,1,1605,2050,2205,1,1,735,252,2,0,727,0,1330,995,2,3,6


In [24]:
target_col = 'NObeyesdad'

X = train.drop([target_col], axis=1) # 目的変数を指定する
y = train[target_col].values # 目的変数を指定する

# 機械学習モデルの構築・学習・予測

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

sample_sub = pd.read_csv(FILE_PATH + "sample_submission.csv")

In [26]:
# 混同行列
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 機械学習モデルのフォーマットを設定
seed = 42

xgb = XGBClassifier(random_state=seed)
lgb = LGBMClassifier(random_state=seed)
cb = CatBoostClassifier(random_state=seed, verbose=0)

submit = False

## ハイパーパラメータ(optuna)

### Catboost

In [37]:
if submit==False:

    import optuna
    from catboost import CatBoostClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    # Assuming 'X' is your feature matrix and 'y' is your target variable
    X = X.copy()
    y = y.copy()

    def objective(trial):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        params = {
            'iterations': trial.suggest_int('iterations', 200, 1000),
            'depth': trial.suggest_int('depth', 3, 10),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 20),
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.2, log=True),
            'random_state': 42,
            'verbose': 0,
#             'eval_metric': 'Accuracy',
        }

        model = CatBoostClassifier(**params)

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50)

        y_pred = model.predict(X_val)
        metrics = model.score(y_val, y_pred)

        return metrics

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    trial = study.best_trial

    print('Value: ', trial.value)
    print('Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

[I 2024-02-01 03:00:12,215] A new study created in memory with name: no-name-0e53cb2d-49ca-4801-ab78-b2f671d31ba9
[I 2024-02-01 03:00:46,086] Trial 0 finished with value: 0.12620423892100194 and parameters: {'iterations': 301, 'depth': 10, 'min_data_in_leaf': 15, 'learning_rate': 0.017886127110244773}. Best is trial 0 with value: 0.12620423892100194.
[I 2024-02-01 03:01:38,035] Trial 1 finished with value: 0.12716763005780346 and parameters: {'iterations': 458, 'depth': 10, 'min_data_in_leaf': 6, 'learning_rate': 0.012184743760914682}. Best is trial 1 with value: 0.12716763005780346.
[I 2024-02-01 03:01:46,976] Trial 2 finished with value: 0.10476878612716763 and parameters: {'iterations': 494, 'depth': 6, 'min_data_in_leaf': 16, 'learning_rate': 0.00015615691623043512}. Best is trial 1 with value: 0.12716763005780346.
[I 2024-02-01 03:02:44,558] Trial 3 finished with value: 0.12620423892100194 and parameters: {'iterations': 861, 'depth': 9, 'min_data_in_leaf': 20, 'learning_rate': 0.0

Number of finished trials:  100
Best trial:
Value:  0.18762042389210018
Params: 
    iterations: 526
    depth: 3
    min_data_in_leaf: 13
    learning_rate: 0.0008029819794568197


### XGBClassifier

In [36]:
if submit == False:
    import optuna
    from xgboost import XGBClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score

    # Assuming 'X' is your feature matrix and 'y' is your target variable
    X = X.copy()
    y = y.copy()

    def objective(trial):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        params = {
            'objective':"multi:softprob",
            'max_depth': trial.suggest_int('max_depth', 5, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 150, 1000),
            'subsample': trial.suggest_float('subsample', 0.01, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
            'random_state': trial.suggest_categorical('random_state', [42]),
            'tree_method': 'hist',  # Use GPU for training
            'device': 'cuda',
            'njobs': -1,
            'eval_metric': 'mlogloss',  # Evaluation metric
            'verbosity': 2,  # Set verbosity to 0 for less output
        }

        model = XGBClassifier(**params)

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)

        y_pred = model.predict(X_val)
        metrics = model.score(y_val, y_pred)

        return metrics

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    trial = study.best_trial

    print('Value: ', trial.value)
    print('Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

[I 2024-02-01 02:56:05,058] A new study created in memory with name: no-name-8f8f839c-647a-4cc9-8cee-5a912c52d537
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [02:56:05] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "njobs" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-02-01 02:56:07,172] Trial 0 finished with value: 0.1317437379576108 and parameters: {'max_depth': 7, 'min_child_weight': 10, 'learning_rate': 0.44541594988725697, 'n_estimators': 184, 'subsample': 0.36384956722782635, 'colsample_bytree': 0.2534756518503573, 'random_state': 42}. Best is trial 0 with value: 0.1317437379576108.
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping

Number of finished trials:  100
Best trial:
Value:  0.1365606936416185
Params: 
    max_depth: 10
    min_child_weight: 1
    learning_rate: 0.733656719765889
    n_estimators: 696
    subsample: 0.7928524918864644
    colsample_bytree: 0.1345360036201174
    random_state: 42


### LGBMClassifier

In [47]:
# if submit == False:
#     import optuna
#     import lightgbm as lgb
#     from sklearn.model_selection import train_test_split
#     from sklearn.metrics import roc_auc_score

#     # Assuming 'X' is your feature matrix and 'y' is your target variable
#     X = X.copy()
#     y = y.copy()

#     def objective(trial):
#         X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#         params = {
#             'boosting_type': 'gbdt',
#             'metric': trial.suggest_categorical('metric', ['multi_logloss']),
#             'max_depth': trial.suggest_int('max_depth', 5, 10),
#             'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
#             'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
#             'n_estimators': trial.suggest_int('n_estimators', 150, 1000),
#             'subsample': trial.suggest_float('subsample', 0.1, 1.0),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
#             'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
#             'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
# #             'num_leaves': 
#             'random_state': 42,
#             'num_class': trial.suggest_categorical('num_class', [7])
#         }

#         model = lgb.LGBMClassifier(**params)

#         model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

#         y_pred = model.predict(X_val)
#         metrics = model.score(y_val, y_pred)

#         return metrics

#     study = optuna.create_study(direction='maximize')
#     study.optimize(objective, n_trials=100)

#     print('Number of finished trials: ', len(study.trials))
#     print('Best trial:')
#     trial = study.best_trial

#     print('Value: ', trial.value)
#     print('Params: ')
#     for key, value in trial.params.items():
#         print(f'    {key}: {value}')

In [51]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, StratifiedKFold, RepeatedStratifiedKFold

def objective_lgb(trial):
    """Define the objective function"""
    params = {
        'metric': trial.suggest_categorical('metric', ['multi_logloss']),
        'num_class': trial.suggest_categorical('num_class', [3]),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        "random_state" : trial.suggest_categorical('random_state', [42]),
    }


    lgb = LGBMClassifier(**params, device = 'gpu', n_jobs = -1)
    cv = abs(cross_val_score(lgb, X_train, y_train, cv = skf,scoring='neg_log_loss').mean())
    return cv

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.85, random_state = 2610, stratify = y)
skf = RepeatedStratifiedKFold(n_splits = 3, n_repeats = 1, random_state = 2610)

In [1]:
# %%time
# study = optuna.create_study(direction='minimize')
# study.optimize(objective_lgb, n_trials=100, timeout=2000)

### パラメータ

In [7]:
# lgb_params = {'max_depth': 7,
#     'min_child_samples': 1,
#     'learning_rate': 0.04163336702197924,
#     'n_estimators': 982,
#     'subsample': 0.10132257615206398,
#     'colsample_bytree': 0.3686160004388141,
#     'reg_alpha': 0.6838448061147125,
#     'reg_lambda': 0.8044499417370345}

xgb_params = {'max_depth': 10,
    'min_child_weight': 1,
    'learning_rate': 0.733656719765889,
    'n_estimators': 696,
    'subsample': 0.7928524918864644,
    'colsample_bytree': 0.1345360036201174,
    'random_state': 42,
    'njobs': -1}

cat_params = {'iterations': 526,
    'depth': 3,
    'min_data_in_leaf': 13,
    'learning_rate': 0.0008029819794568197}

In [8]:
lgb_model = LGBMClassifier(**lgb_params)
xgb_model = XGBClassifier(**xgb_params)
cat_model = CatBoostClassifier(**cat_params)

# 参考（目的変数のエンコーディング）

予測の部分は、参考にした資料が予測確率のため、今回は流用できない

In [ ]:
# df[target_col] = df[target_col].map({
#     "Insufficient_Weight":0,
#     "Normal_Weight":1,
#     "Obesity_Type_I":2,
#     "Obesity_Type_II":3,
#     "Obesity_Type_III":4,
#     "Overweight_Level_I":5,
#     "Overweight_Level_II":6,
#     })
